In [3]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from utils import check_nan, check_unique
from matplotlib import gridspec
from matplotlib import rcParams
from collections import Counter

#Paramètres graphiques 
rcParams['figure.figsize'] = 15,8 #Taille de la figure affichée
sns.set_style("darkgrid")#style de l'arrière plan de seaborn 
sns.set_palette("pastel")#Couleurs utilisées dans les graphiques 
plt.rcParams["figure.dpi"] = 200 #dpi = dot per inch , résolution des graphiques 
pd.set_option('display.max_columns', 500) #Nombre de colonne maximum du dataframe à afficher

# import warnings
# warnings.simplefilter(action='ignore')

# Lecture et mise en forme des données

# Mise en forme des données :

In [4]:
data=pd.read_csv("../../delays.csv")
data.columns= ["save_datetime","id_train_complet","transporteur","date_depart","depart_destination","arrivee_prevue","retard","nom_station"]

data["save_datetime"]=pd.to_datetime(data["save_datetime"])
data["date_depart"]=pd.to_datetime(data["date_depart"])


#Drop des lignes avec une horaire prévue inexploitatble :
df=data[data["arrivee_prevue"]!='Nie dotyczy']

#Conversion du retard en entier : 
df["retard"]=df["retard"].str.split(" ").str[0].astype('int')

new_id=df["id_train_complet"].str.extract(r"(\d+)(\/\d+)?\s*(\(\d+\)|)?(\s*)(.*|[a-zA-Z\u00C0-\u017F]*)").add_prefix('index_part')
df=df.merge(new_id, left_index=True, right_index=True).drop(['index_part3'], axis=1).rename(columns={"index_part4": "index_part3","index_part0":"id_train"})

df=df.fillna("")



# On ne garde que les lignes qui nous intéressent
#df_last =df.sort_values("save_datetime").drop_duplicates(["id_train_complet","transporteur","date_depart","depart_destination","arrivee_prevue","nom_station"],keep="last").sort_values("save_datetime")

df["arrivee_prevue"]=pd.to_datetime(df["arrivee_prevue"], format='%H:%M').dt.time

print("Shape du dataset d'origine :", data.shape)
print("Shape du dataset contenant uniquement les lignes qui nous intéressent : ",df.shape)
df.head()

C:\Users\Nicolas Carval\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


Shape du dataset d'origine : (3718170, 8)
Shape du dataset contenant uniquement les lignes qui nous intéressent :  (3717412, 12)


,save_datetime,id_train_complet,transporteur,date_depart,depart_destination,arrivee_prevue,retard,nom_station,id_train,index_part1,index_part2,index_part3
0,2022-05-16,41002/3 (102) POLONIA,PKP Intercity,2022-05-15,Bohumin - Warszawa Wschodnia,21:31:00,162,Warszawa Zachodnia,41002,/3,(102),POLONIA
1,2022-05-16,91888/9 (102),Koleje Mazowieckie,2022-05-16,Siedlce - Warszawa Zachodnia,00:15:00,0,Warszawa Zachodnia,91888,/9,(102),
2,2022-05-16,21644/5 (102),Koleje Mazowieckie,2022-05-16,Radom Główny - Warszawa Wschodnia,00:29:00,0,Warszawa Zachodnia,21644,/5,(102),
3,2022-05-16,19300/1 (102),Koleje Mazowieckie,2022-05-16,Warszawa Wschodnia - Grodzisk Mazowiecki,00:34:00,0,Warszawa Zachodnia,19300,/1,(102),
4,2022-05-16,38170/1 (102) USTRONIE,PKP Intercity,2022-05-16,Kraków Główny - Kołobrzeg,00:55:00,0,Warszawa Zachodnia,38170,/1,(102),USTRONIE


In [5]:
check_nan(df)
print("\n")
check_unique(df)

Valeurs nan dans save_datetime : 0
Valeurs nan dans id_train_complet : 0
Valeurs nan dans transporteur : 0
Valeurs nan dans date_depart : 0
Valeurs nan dans depart_destination : 0
Valeurs nan dans arrivee_prevue : 0
Valeurs nan dans retard : 0
Valeurs nan dans nom_station : 0
Valeurs nan dans id_train : 0
Valeurs nan dans index_part1 : 0
Valeurs nan dans index_part2 : 0
Valeurs nan dans index_part3 : 0


Valeurs uniques dans save_datetime : 4032
Valeurs uniques dans id_train_complet : 7947
Valeurs uniques dans transporteur : 10
Valeurs uniques dans date_depart : 16
Valeurs uniques dans depart_destination : 995
Valeurs uniques dans arrivee_prevue : 1390
Valeurs uniques dans retard : 358
Valeurs uniques dans nom_station : 95
Valeurs uniques dans id_train : 3447
Valeurs uniques dans index_part1 : 11
Valeurs uniques dans index_part2 : 75
Valeurs uniques dans index_part3 : 335


# Observations sur les id_train  : 

# Feature preprocessing : 

## Extraction de départ - destination : 

In [6]:
df[['depart','destination']] = df["depart_destination"].str.split(" - ",expand=True,)
df.head()

,save_datetime,id_train_complet,transporteur,date_depart,depart_destination,arrivee_prevue,retard,nom_station,id_train,index_part1,index_part2,index_part3,depart,destination
0,2022-05-16,41002/3 (102) POLONIA,PKP Intercity,2022-05-15,Bohumin - Warszawa Wschodnia,21:31:00,162,Warszawa Zachodnia,41002,/3,(102),POLONIA,Bohumin,Warszawa Wschodnia
1,2022-05-16,91888/9 (102),Koleje Mazowieckie,2022-05-16,Siedlce - Warszawa Zachodnia,00:15:00,0,Warszawa Zachodnia,91888,/9,(102),,Siedlce,Warszawa Zachodnia
2,2022-05-16,21644/5 (102),Koleje Mazowieckie,2022-05-16,Radom Główny - Warszawa Wschodnia,00:29:00,0,Warszawa Zachodnia,21644,/5,(102),,Radom Główny,Warszawa Wschodnia
3,2022-05-16,19300/1 (102),Koleje Mazowieckie,2022-05-16,Warszawa Wschodnia - Grodzisk Mazowiecki,00:34:00,0,Warszawa Zachodnia,19300,/1,(102),,Warszawa Wschodnia,Grodzisk Mazowiecki
4,2022-05-16,38170/1 (102) USTRONIE,PKP Intercity,2022-05-16,Kraków Główny - Kołobrzeg,00:55:00,0,Warszawa Zachodnia,38170,/1,(102),USTRONIE,Kraków Główny,Kołobrzeg


In [7]:
travel_dict={}
for i in tqdm(df["id_train_complet"].unique()):
    a=df[df["id_train_complet"]==i].sort_values(["date_depart",'id_train_complet',"arrivee_prevue"])["nom_station"].tolist()
    if Counter(a).most_common()[0][1]-Counter(a).most_common()[-1][1]<2:
        start_station=Counter(a).most_common()[-1][0]
        s=-1
    else:
        start_station=Counter(a).most_common()[-2][0]
        s=-2
    
    if Counter(a).most_common()[s][1]>1:
        start=a.index(start_station)
        stop=[j for j, n in enumerate(a) if n == start_station][1]
        travel_dict[i]=a[start:stop]
    else:
        travel_dict[i]=a
#travel_dict

100%|██████████████████████████████████████████████████████████████████████████████| 7947/7947 [26:53<00:00,  4.92it/s]


In [8]:
df["travel"]=df.apply(lambda x: travel_dict[x["id_train_complet"]], axis=1)
df["num_station"]=df.apply(lambda x: travel_dict[x["id_train_complet"]].index(x["nom_station"])
                            if x["nom_station"] in x["travel"] else -1 , axis=1)

In [10]:
df.to_csv("df_base.csv")

In [11]:
df.head()

,save_datetime,id_train_complet,transporteur,date_depart,depart_destination,arrivee_prevue,retard,nom_station,id_train,index_part1,index_part2,index_part3,depart,destination,travel,num_station
0,2022-05-16,41002/3 (102) POLONIA,PKP Intercity,2022-05-15,Bohumin - Warszawa Wschodnia,21:31:00,162,Warszawa Zachodnia,41002,/3,(102),POLONIA,Bohumin,Warszawa Wschodnia,[Warszawa Zachodnia],0
1,2022-05-16,91888/9 (102),Koleje Mazowieckie,2022-05-16,Siedlce - Warszawa Zachodnia,00:15:00,0,Warszawa Zachodnia,91888,/9,(102),,Siedlce,Warszawa Zachodnia,[Warszawa Zachodnia],0
2,2022-05-16,21644/5 (102),Koleje Mazowieckie,2022-05-16,Radom Główny - Warszawa Wschodnia,00:29:00,0,Warszawa Zachodnia,21644,/5,(102),,Radom Główny,Warszawa Wschodnia,[Warszawa Wschodnia],-1
3,2022-05-16,19300/1 (102),Koleje Mazowieckie,2022-05-16,Warszawa Wschodnia - Grodzisk Mazowiecki,00:34:00,0,Warszawa Zachodnia,19300,/1,(102),,Warszawa Wschodnia,Grodzisk Mazowiecki,[Warszawa Zachodnia],0
4,2022-05-16,38170/1 (102) USTRONIE,PKP Intercity,2022-05-16,Kraków Główny - Kołobrzeg,00:55:00,0,Warszawa Zachodnia,38170,/1,(102),USTRONIE,Kraków Główny,Kołobrzeg,[Warszawa Wschodnia],-1


In [12]:
df_True =df.sort_values("save_datetime").drop_duplicates(["id_train_complet","transporteur","date_depart","depart_destination","arrivee_prevue","nom_station"],keep="last").sort_values("save_datetime")
df_Predicted = df[~df.index.isin(df_True.index)]

In [12]:
df[df["id_train_complet"]=="95697  "].sort_values(["date_depart",'num_station']).iloc[:30]

,save_datetime,id_train_complet,transporteur,date_depart,depart_destination,arrivee_prevue,retard,nom_station,id_train,index_part1,index_part2,index_part3,depart,destination,datetime_save,travel,num_station
175,2022-05-16 00:00:00,95697,PKP SKM w Trójmieście Sp. z o.o.,2022-05-15,Gdańsk Zaspa - Wejherowo,23:38:00,24,Reda Pieleszewo,95697,,,,Gdańsk Zaspa,Wejherowo,2022-05-16 00:00:00,"[Reda, Reda Pieleszewo, Wejherowo Śmiechowo, W...",1
373,2022-05-16 00:05:00,95697,PKP SKM w Trójmieście Sp. z o.o.,2022-05-15,Gdańsk Zaspa - Wejherowo,23:47:00,23,Wejherowo,95697,,,,Gdańsk Zaspa,Wejherowo,2022-05-16 00:05:00,"[Reda, Reda Pieleszewo, Wejherowo Śmiechowo, W...",4
411,2022-05-16 00:05:00,95697,PKP SKM w Trójmieście Sp. z o.o.,2022-05-15,Gdańsk Zaspa - Wejherowo,23:41:00,24,Wejherowo Śmiechowo,95697,,,,Gdańsk Zaspa,Wejherowo,2022-05-16 00:05:00,"[Reda, Reda Pieleszewo, Wejherowo Śmiechowo, W...",2
408,2022-05-16 00:05:00,95697,PKP SKM w Trójmieście Sp. z o.o.,2022-05-15,Gdańsk Zaspa - Wejherowo,23:44:00,23,Wejherowo-Nanice,95697,,,,Gdańsk Zaspa,Wejherowo,2022-05-16 00:05:00,"[Reda, Reda Pieleszewo, Wejherowo Śmiechowo, W...",3
266818,2022-05-16 22:30:00,95697,PKP SKM w Trójmieście Sp. z o.o.,2022-05-16,Gdańsk Śródmieście - Wejherowo,22:34:00,0,Gdańsk Główny,95697,,,,Gdańsk Śródmieście,Wejherowo,2022-05-16 22:30:00,"[Reda, Reda Pieleszewo, Wejherowo Śmiechowo, W...",5
267587,2022-05-16 22:35:00,95697,PKP SKM w Trójmieście Sp. z o.o.,2022-05-16,Gdańsk Śródmieście - Wejherowo,22:39:00,0,Gdańsk Politechnika,95697,,,,Gdańsk Śródmieście,Wejherowo,2022-05-16 22:35:00,"[Reda, Reda Pieleszewo, Wejherowo Śmiechowo, W...",7
267583,2022-05-16 22:35:00,95697,PKP SKM w Trójmieście Sp. z o.o.,2022-05-16,Gdańsk Śródmieście - Wejherowo,22:36:00,0,Gdańsk Stocznia,95697,,,,Gdańsk Śródmieście,Wejherowo,2022-05-16 22:35:00,"[Reda, Reda Pieleszewo, Wejherowo Śmiechowo, W...",6
268018,2022-05-16 22:40:00,95697,PKP SKM w Trójmieście Sp. z o.o.,2022-05-16,Gdańsk Śródmieście - Wejherowo,22:41:00,0,Gdańsk Wrzeszcz,95697,,,,Gdańsk Śródmieście,Wejherowo,2022-05-16 22:40:00,"[Reda, Reda Pieleszewo, Wejherowo Śmiechowo, W...",8
268203,2022-05-16 22:40:00,95697,PKP SKM w Trójmieście Sp. z o.o.,2022-05-16,Gdańsk Śródmieście - Wejherowo,22:44:00,0,Gdańsk Zaspa,95697,,,,Gdańsk Śródmieście,Wejherowo,2022-05-16 22:40:00,"[Reda, Reda Pieleszewo, Wejherowo Śmiechowo, W...",9
268778,2022-05-16 22:45:00,95697,PKP SKM w Trójmieście Sp. z o.o.,2022-05-16,Gdańsk Śródmieście - Wejherowo,22:46:00,0,Gdańsk Przymorze-Uniwersytet,95697,,,,Gdańsk Śródmieście,Wejherowo,2022-05-16 22:45:00,"[Reda, Reda Pieleszewo, Wejherowo Śmiechowo, W...",10


In [78]:
discriminant = ["id_train","transporteur","date_depart","depart_destination","arrivee_prevue","nom_station"]

In [44]:
[]

"['id_train_complet', 'transporteur', 'date_depart', 'depart_destination', 'arrivee_prevue', 'nom_station']"

In [14]:
df_Predicted["AE"] = 0

C:\Users\Nicolas Carval\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [80]:
df_True.head()

,save_datetime,id_train_complet,transporteur,date_depart,depart_destination,arrivee_prevue,retard,nom_station,id_train,index_part1,index_part2,index_part3,depart,destination,travel,num_station,id_unique_True,id_uniqu_True
175,2022-05-16,95697,PKP SKM w Trójmieście Sp. z o.o.,2022-05-15,Gdańsk Zaspa - Wejherowo,23:38:00,24,Reda Pieleszewo,95697,,,,Gdańsk Zaspa,Wejherowo,[Gdańsk Stocznia],-1,175 95697\n68 91888\n111 ...,175 95697\n68 91888\n111 ...
68,2022-05-16,91888/9,Koleje Mazowieckie,2022-05-16,Siedlce - Warszawa Zachodnia,00:01:00,2,Warszawa Stadion,91888,/9,,,Siedlce,Warszawa Zachodnia,[Warszawa Wschodnia],-1,175 95697\n68 91888\n111 ...,175 95697\n68 91888\n111 ...
111,2022-05-16,59486,PKP SKM w Trójmieście Sp. z o.o.,2022-05-15,Gdynia Cisowa - Gdańsk Główny,23:54:00,9,Gdańsk Żabianka-AWFiS,59486,,,,Gdynia Cisowa,Gdańsk Główny,[Gdynia Chylonia],-1,175 95697\n68 91888\n111 ...,175 95697\n68 91888\n111 ...
119,2022-05-16,95699,PKP SKM w Trójmieście Sp. z o.o.,2022-05-16,Gdańsk Śródmieście - Wejherowo,00:04:00,0,Gdynia Wzgórze Św.Maksymiliana,95699,,,,Gdańsk Śródmieście,Wejherowo,[Gdańsk Główny],-1,175 95697\n68 91888\n111 ...,175 95697\n68 91888\n111 ...
116,2022-05-16,95699,PKP SKM w Trójmieście Sp. z o.o.,2022-05-16,Gdańsk Śródmieście - Wejherowo,00:01:00,0,Gdynia Redłowo,95699,,,,Gdańsk Śródmieście,Wejherowo,[Gdańsk Główny],-1,175 95697\n68 91888\n111 ...,175 95697\n68 91888\n111 ...


In [82]:
#df_Predicted["id_unique"] = str(df_Predicted["id_train"])+str(df_Predicted["transporteur"])+str(df_Predicted["date_depart"])+str(df_Predicted["depart_destination"])+ str(df_Predicted["arrivee_prevue"])+ str(df_Predicted["nom_station"])
#df_True["id_unique_True"] = 


df_True.apply(lambda row:str(row["id_train"])+str(row["transporteur"])+str(row["date_depart"])+str(row["depart_destination"])+ str(row["arrivee_prevue"])+ str(row["nom_station"]))

KeyError: 'id_train'

In [73]:
len(df_True["id_unique_True"].unique())

1

In [71]:
for idi in df_True["id_unique_True"].unique():
    val = df_True[df_True["id_unique_True"]==idi]["retard"]
    val2 = df_Predicted.loc[df_Predicted["id_unique"]==idi,"retard_true"].shape[0]
    print(val)
    print(val2)
    arr = np.full(shape=val2, fill_value  = val)
    df_Predicted.loc[df_Predicted["id_unique"]==idi,"retard_true"] = arr

175        24
68          2
111         9
119         0
116         0
           ..
3718159     4
3718160     9
3718161     0
3718155     3
3718169     0
Name: retard, Length: 264141, dtype: int32
0


ValueError: could not broadcast input array from shape (264141,) into shape (0,)

In [51]:
def get_AE(row):
    print("h")
    return np.abs(row["retard"]-row["retard_true"]) #df_True[df_True[discriminant]==row[discriminant]]["retard"].iloc[0])

In [62]:
df_Predicted.head()

,save_datetime,id_train_complet,transporteur,date_depart,depart_destination,arrivee_prevue,retard,nom_station,id_train,index_part1,index_part2,index_part3,depart,destination,travel,num_station,AE,id_unique,retard_true
0,2022-05-16,41002/3 (102) POLONIA,PKP Intercity,2022-05-15,Bohumin - Warszawa Wschodnia,21:31:00,162,Warszawa Zachodnia,41002,/3,(102),POLONIA,Bohumin,Warszawa Wschodnia,[Warszawa Zachodnia],0,0,0 41002\n1 91888\n2 ...,NaN
1,2022-05-16,91888/9 (102),Koleje Mazowieckie,2022-05-16,Siedlce - Warszawa Zachodnia,00:15:00,0,Warszawa Zachodnia,91888,/9,(102),,Siedlce,Warszawa Zachodnia,[Warszawa Zachodnia],0,0,0 41002\n1 91888\n2 ...,NaN
2,2022-05-16,21644/5 (102),Koleje Mazowieckie,2022-05-16,Radom Główny - Warszawa Wschodnia,00:29:00,0,Warszawa Zachodnia,21644,/5,(102),,Radom Główny,Warszawa Wschodnia,[Warszawa Wschodnia],-1,0,0 41002\n1 91888\n2 ...,NaN
3,2022-05-16,19300/1 (102),Koleje Mazowieckie,2022-05-16,Warszawa Wschodnia - Grodzisk Mazowiecki,00:34:00,0,Warszawa Zachodnia,19300,/1,(102),,Warszawa Wschodnia,Grodzisk Mazowiecki,[Warszawa Zachodnia],0,0,0 41002\n1 91888\n2 ...,NaN
4,2022-05-16,38170/1 (102) USTRONIE,PKP Intercity,2022-05-16,Kraków Główny - Kołobrzeg,00:55:00,0,Warszawa Zachodnia,38170,/1,(102),USTRONIE,Kraków Główny,Kołobrzeg,[Warszawa Wschodnia],-1,0,0 41002\n1 91888\n2 ...,NaN


In [52]:
df_Predicted.apply(lambda row: get_AE(row), axis=1)

KeyboardInterrupt: 

In [ ]:
df_Predicted.head(10)

In [ ]:
# retard N station précédentes
# Trafic moyen
# Jour de la semaine
# Heure de la journée
# retard jour précédent même trajet même station
# retard moyen jour précédent même trajet
# durée de trajet prévue entre les 2 gares (sans retard)

In [ ]:
# At station N, predict delay N+1 N+2 ... N+K